In [1]:
import pandas as pd
import numpy as np
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as Fq
from pyspark.sql.window import Window
from pyspark.sql.functions import countDistinct

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
%%time
# spark session
os.environ['SPARK_HOME'] = "/opt/cloudera/parcels/CDH/lib/spark/"
os.environ['PYSPARK_PYTHON'] = '/opt/cloudera/parcels/Anaconda/envs/py36/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/cloudera/parcels/Anaconda/envs/py36/bin/python'

spark = SparkSession.builder \
    .master("yarn") \
    .appName("test") \
    .enableHiveSupport() \
    .config("spark.submit.deployMode","client") \
    .config("spark.yarn.appMasterEnv.PYSPARK_PYTHON","/opt/cloudera/parcels/Anaconda/envs/py36/bin/python") \
    .config("spark.yarn.appMasterEnv.PYSPARK_DRIVER_PYTHON","/opt/cloudera/parcels/Anaconda/envs/py36/bin/python") \
    .config("spark.driver.memory","8g") \
    .config("spark.executor.memory","16g") \
    .getOrCreate()
print(spark)

CPU times: user 21.1 ms, sys: 22.5 ms, total: 43.6 ms
Wall time: 32.3 s


In [4]:
from datetime import datetime, timedelta

def generate_dates():
    start_date = datetime.strptime('2022-08-01', '%Y-%m-%d')
    end_date = datetime.strptime('2023-09-01', '%Y-%m-%d')
    current_date = start_date
    date_list = []

    while current_date <= end_date:
        date_list.append(current_date.strftime('%Y-%m-%d'))
        # Increment by one month
        if current_date.month == 12:
            current_date = current_date.replace(year=current_date.year+1, month=1)
        else:
            current_date = current_date.replace(month=current_date.month+1)

    return date_list

dates = generate_dates()

tables = ['exp_clr_balance_v2', 'exp_data_v2', 'exp_device_v2', 'exp_interco_v2', 'exp_prep_arpu_v2', 'exp_parc_prep_v2', 
          'exp_pass_v2', 'exp_rech_v2', 'exp_roam_v2', 'exp_vas_v2', 'exp_voix_v2']

In [5]:
dates

['2022-08-01',
 '2022-09-01',
 '2022-10-01',
 '2022-11-01',
 '2022-12-01',
 '2023-01-01',
 '2023-02-01',
 '2023-03-01',
 '2023-04-01',
 '2023-05-01',
 '2023-06-01',
 '2023-07-01',
 '2023-08-01',
 '2023-09-01']

In [6]:
k = 3 

date_start = dates[k]
date_end = dates[k+1]
date_final_obs = dates[k+3]
date_data_origin = dates[k-3]

In [7]:
l = [date_start, date_end, date_final_obs, date_data_origin]
l

['2022-11-01', '2022-12-01', '2023-02-01', '2022-08-01']

In [8]:
exp_sdp_remboursement_recharge_v2 = spark.sql(f"""SELECT * 
FROM 
tel_test_avrd.exp_sdp_remboursement_recharge_v2 
WHERE event_time >= '{date_start}' AND event_time < '{date_final_obs}'""")
exp_sdp_remboursement_recharge_v2.show(n=2, vertical=True)

-RECORD 0------------------------------------
 final_anon_mdn       | 29010156965826191... 
 event_time           | 2023-01-16 22:40:02  
 profile_id           | 84001                
 segment              | B2C                  
 formule              | Prepaid              
 desc_profil          | TicTac 4G            
 produit              | TicTac 4G            
 gamme                | TicTac               
 marche               | Mobile Prepaid       
 balance_debit_before | 200                  
 balance_debit_after  | 0                    
 account_type_id_1    | 5121                 
 bal1_descr           | DATA                 
 bal1_unit            | MO                   
 bal1_revenue_flag    | N                    
 account_type_id_2    | 5201                 
 bal2_descr           | Report DATA          
 bal2_unit            | MO                   
 bal2_revenue_flag    | N                    
 account_type_id_3    |                      
 bal3_descr           | null      

In [9]:
exp_sdp_avr_v2 = spark.sql(f"""SELECT * 
    FROM tel_test_avrd.exp_sdp_avr_v2 
    WHERE timestamp >= '{date_start}' AND timestamp < '{date_end}' 
    ORDER BY RAND() LIMIT 100000""")
exp_sdp_avr_v2.show(n=2, vertical=True)

Py4JJavaError: An error occurred while calling o120.showString.
: org.apache.spark.SparkException: Job 3 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:935)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:933)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:933)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:2129)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2042)
	at org.apache.spark.SparkContext$$anonfun$stop$7.apply$mcV$sp(SparkContext.scala:1963)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1388)
	at org.apache.spark.SparkContext.stop(SparkContext.scala:1962)
	at org.apache.spark.

In [ ]:
avr_reimbursement = exp_sdp_avr_v2.join(exp_sdp_remboursement_recharge_v2, 'final_anon_mdn', how='left').select('final_anon_mdn', 'timestamp', 'event_time').toPandas()

In [11]:
avr_reimbursement.head(10)

,final_anon_mdn,timestamp,event_time
0,0009199021916181057050608,2022-11-08 09:38:12.0,2022-12-02 19:58:09
1,0009199021916181057050608,2022-11-08 09:38:12.0,2022-11-07 20:41:35
2,0009199021916181057050608,2022-11-08 09:38:12.0,2022-12-30 18:43:51
3,0009199021916181057050608,2022-11-08 09:38:12.0,2022-11-20 16:05:31
4,0009199021916181057050608,2022-11-08 09:38:12.0,2022-11-16 16:41:17
5,0009199021916181057050608,2022-11-08 09:38:12.0,2022-12-08 11:12:38
6,0009199021916181057050608,2022-11-08 09:38:12.0,2022-12-27 13:52:08
7,0009199021916181057050608,2022-11-08 09:38:12.0,2022-12-05 14:22:23
8,0009199021916181057050608,2022-11-08 09:38:12.0,2022-11-10 21:26:31
9,0009199021916181057050608,2022-11-08 09:38:12.0,2022-12-16 16:18:18


In [13]:
len(avr_reimbursement)

988061

In [12]:
avr_reimbursement['timestamp'] = pd.to_datetime(avr_reimbursement['timestamp'])
avr_reimbursement['reinbursement_time'] = avr_reimbursement.apply(
    lambda row: date_final_obs if (pd.isnull(row['event_time']) or row['event_time'] < row['timestamp']) 
    else row['event_time'], axis=1)
avr_reimbursement.head(20)

,final_anon_mdn,timestamp,event_time,reinbursement_time
0,0009199021916181057050608,2022-11-08 09:38:12,2022-12-02 19:58:09,2022-12-02 19:58:09
1,0009199021916181057050608,2022-11-08 09:38:12,2022-11-07 20:41:35,2023-02-01 00:00:00
2,0009199021916181057050608,2022-11-08 09:38:12,2022-12-30 18:43:51,2022-12-30 18:43:51
3,0009199021916181057050608,2022-11-08 09:38:12,2022-11-20 16:05:31,2022-11-20 16:05:31
4,0009199021916181057050608,2022-11-08 09:38:12,2022-11-16 16:41:17,2022-11-16 16:41:17
5,0009199021916181057050608,2022-11-08 09:38:12,2022-12-08 11:12:38,2022-12-08 11:12:38
6,0009199021916181057050608,2022-11-08 09:38:12,2022-12-27 13:52:08,2022-12-27 13:52:08
7,0009199021916181057050608,2022-11-08 09:38:12,2022-12-05 14:22:23,2022-12-05 14:22:23
8,0009199021916181057050608,2022-11-08 09:38:12,2022-11-10 21:26:31,2022-11-10 21:26:31
9,0009199021916181057050608,2022-11-08 09:38:12,2022-12-16 16:18:18,2022-12-16 16:18:18


In [19]:
avr_reimbursement['date_diff'] = pd.to_datetime(avr_reimbursement['reinbursement_time']) - pd.to_datetime(avr_reimbursement['timestamp'])
avr_reimbursement['date_diff'] = avr_reimbursement['date_diff'].dt.days
avr_reimbursement['flag'] = (avr_reimbursement['date_diff'] <= 45).astype(int)

In [20]:
avr_reimbursement.head(70)

,final_anon_mdn,timestamp,event_time,reinbursement_time,date_diff,flag
0,0009199021916181057050608,2022-11-08 09:38:12,2022-12-02 19:58:09,2022-12-02 19:58:09,24,1
1,0009199021916181057050608,2022-11-08 09:38:12,2022-11-07 20:41:35,2023-02-01 00:00:00,84,0
2,0009199021916181057050608,2022-11-08 09:38:12,2022-12-30 18:43:51,2022-12-30 18:43:51,52,0
3,0009199021916181057050608,2022-11-08 09:38:12,2022-11-20 16:05:31,2022-11-20 16:05:31,12,1
4,0009199021916181057050608,2022-11-08 09:38:12,2022-11-16 16:41:17,2022-11-16 16:41:17,8,1
5,0009199021916181057050608,2022-11-08 09:38:12,2022-12-08 11:12:38,2022-12-08 11:12:38,30,1
6,0009199021916181057050608,2022-11-08 09:38:12,2022-12-27 13:52:08,2022-12-27 13:52:08,49,0
7,0009199021916181057050608,2022-11-08 09:38:12,2022-12-05 14:22:23,2022-12-05 14:22:23,27,1
8,0009199021916181057050608,2022-11-08 09:38:12,2022-11-10 21:26:31,2022-11-10 21:26:31,2,1
9,0009199021916181057050608,2022-11-08 09:38:12,2022-12-16 16:18:18,2022-12-16 16:18:18,38,1


In [21]:
avr_reimbursement = avr_reimbursement[['final_anon_mdn', 'flag']].groupby('final_anon_mdn').max().reset_index()
avr_reimbursement.head(20)

,final_anon_mdn,flag
0,0000114774094661096041701,1
1,0000141208700050020071451,1
2,0000148619366931089070772,0
3,0000171673299210037031363,1
4,0000173671915411081020422,1
5,0000175041587290050061023,0
6,0000178886770511099080890,1
7,0000193047229701009020370,1
8,0001111643760381032040938,0
9,0001121018354190093090178,1


In [24]:
mdns = tuple(avr_reimbursement['final_anon_mdn'].tolist())

In [25]:
for table in tables:
    date_start_reformat = date_start.replace('-', '')
    date_data_origin_reformat = date_data_origin.replace('-', '')

    table_desc = spark.sql(f"DESCRIBE tel_test_avrd.{table}")
    col_list = [col for col in table_desc.toPandas()['col_name'] if col not in {'id_date', '# col_name', 
                                                                                '# Partition Information', 'final_anon_mdn', 
                                                                                'id_date'}]

    variables = ""
    for col in col_list:
        variables += f", SUM({col}) AS sum_{col}, MAX({col}) AS max_{col}, MIN({col}) AS min_{col}, VARIANCE({col}) AS var_{col}, SUM(CASE WHEN {col} > 0 THEN 1 ELSE 0 END) AS cnt_{col}"
    
    query = f"""SELECT final_anon_mdn {variables}, DATEDIFF('{date_start}', CONCAT(SUBSTRING(MAX(id_date), 1, 4), '-', SUBSTRING(MAX(id_date), 5, 2), '-', SUBSTRING(MAX(id_date), 7, 2))) AS recency_{table} 
    FROM tel_test_avrd.{table} 
    WHERE id_date < '{date_start_reformat}' AND id_date >= '{date_data_origin_reformat}' AND final_anon_mdn IN {mdns} 
    GROUP BY final_anon_mdn"""
    data = spark.sql(query).toPandas()
    avr_reimbursement = avr_reimbursement.merge(data, on="final_anon_mdn", how='left')

Py4JJavaError: An error occurred while calling o165.collectToPython.
: org.apache.spark.SparkException: Job 8 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:935)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:933)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:933)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:2129)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2042)
	at org.apache.spark.SparkContext$$anonfun$stop$7.apply$mcV$sp(SparkContext.scala:1963)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1388)
	at org.apache.spark.SparkContext.stop(SparkContext.scala:1962)
	at org.apache.s

In [4]:
%%time
k = 3 

date_start = dates[k]
date_end = dates[k+1]
date_final_obs = dates[k+3]
date_data_origin = dates[k-3]

exp_sdp_remboursement_recharge_v2 = spark.sql(f"""SELECT * 
FROM 
tel_test_avrd.exp_sdp_remboursement_recharge_v2 
WHERE event_time >= '{date_start}' AND event_time < '{date_final_obs}'""")

exp_sdp_avr_v2 = spark.sql(f"""SELECT * 
FROM tel_test_avrd.exp_sdp_avr_v2 
WHERE timestamp >= '{date_start}' AND timestamp < '{date_end}' 
ORDER BY RAND() LIMIT 100000""")

avr_reimbursement = exp_sdp_avr_v2.join(exp_sdp_remboursement_recharge_v2, 'final_anon_mdn', how='left').select('final_anon_mdn', 'timestamp', 'event_time').toPandas()

avr_reimbursement['timestamp'] = pd.to_datetime(avr_reimbursement['timestamp'])
avr_reimbursement['reinbursement_time'] = avr_reimbursement.apply(
    lambda row: date_final_obs if (pd.isnull(row['event_time']) or row['event_time'] < row['timestamp']) 
    else row['event_time'], axis=1)

avr_reimbursement['date_diff'] = pd.to_datetime(avr_reimbursement['reinbursement_time']) - pd.to_datetime(avr_reimbursement['timestamp'])
avr_reimbursement['date_diff'] = avr_reimbursement['date_diff'].dt.days
avr_reimbursement['flag'] = (avr_reimbursement['date_diff'] <= 45).astype(int)
avr_reimbursement = avr_reimbursement[['final_anon_mdn', 'flag']].groupby('final_anon_mdn').max().reset_index()

mdns = tuple(avr_reimbursement['final_anon_mdn'].tolist())

for table in tables:
    date_start_reformat = date_start.replace('-', '')
    date_data_origin_reformat = date_data_origin.replace('-', '')

    table_desc = spark.sql(f"DESCRIBE tel_test_avrd.{table}")
    col_list = [col for col in table_desc.toPandas()['col_name'] if col not in {'id_date', '# col_name', '# Partition Information', 'final_anon_mdn', 'id_date'}]

    variables = ""
    for col in col_list:
        variables += f", SUM({col}) AS sum_{col}, MAX({col}) AS max_{col}, MIN({col}) AS min_{col}, VARIANCE({col}) AS var_{col}, SUM(CASE WHEN {col} > 0 THEN 1 ELSE 0 END) AS cnt_{col}"
    
    query = f"""SELECT final_anon_mdn {variables}, DATEDIFF('{date_start}', CONCAT(SUBSTRING(MAX(id_date), 1, 4), '-', SUBSTRING(MAX(id_date), 5, 2), '-', SUBSTRING(MAX(id_date), 7, 2))) AS recency_{table} 
    FROM tel_test_avrd.{table} 
    WHERE id_date < '{date_start_reformat}' AND id_date >= '{date_data_origin_reformat}' AND final_anon_mdn IN {mdns} 
    GROUP BY final_anon_mdn"""
    data = spark.sql(query).toPandas()
    avr_reimbursement = avr_reimbursement.merge(data, on="final_anon_mdn", how='left')

query = f"""SELECT 
final_anon_mdn, 
COUNT(1) AS cnt_req_avr, 
SUM(CASE WHEN reason_of_failure = 'Success' AND grantedadvance IS NOT NULL THEN 1 ELSE 0 END) AS cnt_success_avr 
FROM tel_test_avrd.exp_sdp_avr_v2 
WHERE timestamp < '{date_start}' AND final_anon_mdn IN {mdns} 
GROUP BY final_anon_mdn"""

avr = spark.sql(query).toPandas()
avr_reimbursement = avr_reimbursement.merge(avr, on="final_anon_mdn", how='left')

file_name = f'data_{date_start}.csv'
avr_reimbursement.to_csv(file_name, index=False)

Py4JJavaError: An error occurred while calling o165.collectToPython.
: org.apache.spark.SparkException: Job 7 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:935)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:933)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:933)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:2129)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2042)
	at org.apache.spark.SparkContext$$anonfun$stop$7.apply$mcV$sp(SparkContext.scala:1963)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1388)
	at org.apache.spark.SparkContext.stop(SparkContext.scala:1962)
	at org.apache.s